In [348]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from apyori import apriori
from statistics import mean,stdev

In [43]:
# Read the transaction data and subset for transactions in Idaho
chunks = pd.read_csv("DillardsPOS/newTrans.csv", chunksize=10**6)

dat_chunks = []
for chunk in chunks:
    mask = (chunk['store']==3409) | (chunk['store']==3609) | (chunk['store']==3509)
    dat_chunks.append(chunk.loc[mask])

trnsDF = pd.concat(dat_chunks,axis = 0)
trnsDF = trnsDF[trnsDF.type=='P']
trnsDF.drop(columns = ['type'], inplace = True)
del dat_chunks

In [266]:
# Subset dataframe by storesID 
ID1 = trnsDF[trnsDF.store == 3409]
ID2 = trnsDF[trnsDF.store == 3509]
ID3 = trnsDF[trnsDF.store == 3609]

In [270]:
# Format list for apriori
S1 = ID1.groupby(['reg','trans','date'])['sku'].transform(lambda x: ','.join(map(str,x))).to_frame('itemList')
S2 = ID2.groupby(['reg','trans','date'])['sku'].transform(lambda x: ','.join(map(str,x))).to_frame('itemList')
S3 = ID3.groupby(['reg','trans','date'])['sku'].transform(lambda x: ','.join(map(str,x))).to_frame('itemList')

S1 = S1.reset_index(drop = True)
S2 = S2.reset_index(drop = True)
S2 = S2.reset_index(drop = True)

S1 = S1['itemList'].str.split(',')
S2 = S2['itemList'].str.split(',')
S3 = S3['itemList'].str.split(',')

itemLists1 = []
itemLists2 = []
itemLists3 = []

for i in range(len(S1)):
    itemLists1.append(S1[i])
    
for j in range(len(S2)):
    itemLists2.append(S2[j])
    
for k in range(len(S3)):
    itemLists3.append(S3[k])

In [334]:
# Data Exploration

# Determine number of unique sku's
stores = [3409, 3509, 3609]
IDs = [ID1,ID2,ID3]
uCount = []
for i in range(3):
    uCount.append(IDs[i]['sku'].nunique())

for i in range(3):
    p = f'Store {stores[i]} has {uCount[i]} unique sku'
    print(p)
    
# Determine average items/basket
aItems = [[],[],[]]

for i in range(len(itemLists1)):
    aItems[0].append(len(itemLists1[i]))

for i in range(len(itemLists2)):
    aItems[1].append(len(itemLists2[i]))

for i in range(len(itemLists3)):
    aItems[2].append(len(itemLists3[i]))

print('\n')
for i in range(3):
    p = f'Store {stores[i]} has avg {mean(aItems[i])} items/basket'
    print(p)

Store 3409 has 113768 unique sku
Store 3509 has 51030 unique sku
Store 3609 has 64077 unique sku


Store 3409 has avg 3.1436352309472246 items/basket
Store 3509 has avg 3.5852746877717014 items/basket
Store 3609 has avg 3.3766724391177867 items/basket


In [353]:
# Run apriori
aRules1 = apriori(itemLists1,min_support=.0005,min_confidence=.02, min_lift=3,min_length=2)
aResults1 = list(aRules1)

aRules2 = apriori(itemLists2,min_support=.0005,min_confidence=.02, min_lift=3,min_length=2)
aResults2 = list(aRules2)

aRules3 = apriori(itemLists3,min_support=.0005,min_confidence=.02, min_lift=3,min_length=2)
aResults3 = list(aRules3)

In [375]:
# Create interpretable dataframe
header={'store':[],'antecedent':[],'consequent':[],'lift':[]}
finalDF = pd.DataFrame(header)

for item in aResults1:
    pair = item[0] 
    items = [x for x in pair]
    newRow = pd.DataFrame({'store':[3409],'antecedent':[items[0]],'consequent':[items[1]],'lift':[item[2][0][3]]})
    finalDF = finalDF.append(newRow)

for item in aResults2:
    pair = item[0] 
    items = [x for x in pair]
    newRow = pd.DataFrame({'store':[3509],'antecedent':[items[0]],'consequent':[items[1]],'lift':[item[2][0][3]]})
    finalDF = finalDF.append(newRow)

for item in aResults3:
    pair = item[0] 
    items = [x for x in pair]
    newRow = pd.DataFrame({'store':[3609],'antecedent':[items[0]],'consequent':[items[1]],'lift':[item[2][0][3]]})
    finalDF = finalDF.append(newRow)
    
finalDF.sort_values('lift',ascending = False,inplace=True)
finalDF = finalDF.reset_index(drop = True)

In [359]:
# Interpret results
print('Summary statistics by Lift')
grp1 = finalDF.groupby('store')
print(grp1.describe(percentiles=[]))

print('\n Summary statistics for top 100 rules ranked by Lift')
grp2 = finalDF[0:100].groupby('store')
print(grp2.describe(percentiles=[]))

Summary statistics by Lift
        lift                                                            
       count         mean         std       min         50%          max
store                                                                   
3409    28.0    23.225808   23.968440  5.033627    8.539068    83.212486
3509   366.0  1134.649650  414.147775  9.673013  984.536000  1984.951613
3609    45.0   690.899841  556.276117  9.245482  851.845180  1605.220183

 Summary statistics for top 100 rules ranked by Lift
       lift                                                               
      count         mean        std          min          50%          max
store                                                                     
3509   99.0  1642.490263  62.205883  1598.272727  1640.893333  1984.951613
3609    1.0  1605.220183        NaN  1605.220183  1605.220183  1605.220183


In [411]:
newDF = pd.DataFrame(header)
SKU = []

for i in range(len(finalDF)):
    if (finalDF.consequent[i] not in SKU):
        newDF = newDF.append(finalDF.iloc[i])
        SKU.append(finalDF.antecedent[i])
        SKU.append(finalDF.consequent[i])

print('Summary statistics for unique rules')
newDF = newDF.reset_index(drop = True)
grp3 = newDF.groupby('store')
print(grp3.describe(percentiles=[]))

Summary statistics for unique rules
        lift                                                              
       count         mean         std        min          50%          max
store                                                                     
3409.0   4.0    23.023933   29.202014   7.771379     8.756059    66.812236
3509.0  57.0  1171.531087  481.213519  14.828266  1274.289924  1984.951613
3609.0  24.0   798.944492  530.213599   9.768457  1008.725759  1567.254529


In [362]:
finalDF.to_csv("rulesRanked.csv")
newDF.to_csv("rulesRankedUnique.csv")